Import all - data, libraries ect. 

In [34]:
import numpy as np
import pandas as pd

data = pd.read_csv('weekly_data.csv', delimiter = ',', names = ['week','price'])

print(data)

     week   price
0       1  202.64
1       2  166.81
2       3  181.09
3       4  157.53
4       1  170.84
..    ...     ...
245     2  178.00
246     3  211.72
247     4  212.36
248     1  266.86
249     2  232.53

[250 rows x 2 columns]


1. Task - Combine two point forecasts -> Naive and Simple linear regression. Then calculate the coverage of new obtained forecast. Combine them in two ways:

a) Make average after calculating Naive and after making linear regression part. Then based on new obtained forecast calculate coverage.

b) Calculate forecasts then make average of obtained lower and upper bounds from both forecasts. Then based on that calculate coverage.

# Naive forecasts:

a) Naive forecast - shift 3 weeks 

In [35]:
naive1 = pd.DataFrame()
naive1['price'] = data['price'].copy().shift(3)
print(naive1)

      price
0       NaN
1       NaN
2       NaN
3    202.64
4    166.81
..      ...
245  160.62
246  171.64
247  196.73
248  178.00
249  211.72

[250 rows x 1 columns]


# Regression part

b) Simple linear regression part

Consider two autoregressive model:

MODEL2: $P_w = beta_0 + beta_1*P_{w-1} + beta_2*P_{w-2} + beta_3*P_{w-3} + epsilon$

In [36]:
data2 = data.copy()
Pw1 = data2['price'].shift(1)
Pw3 = data2['price'].shift(3)
Pw2 = data2['price'].shift(2)

concated = pd.concat([Pw1,Pw2, Pw3], axis=1)
concated.insert(0, 'intercept', 1)
yData = data2['price']

print(concated)

concatedarray = concated.to_numpy()

print(concatedarray)
print(yData)


     intercept   price   price   price
0            1     NaN     NaN     NaN
1            1  202.64     NaN     NaN
2            1  166.81  202.64     NaN
3            1  181.09  166.81  202.64
4            1  157.53  181.09  166.81
..         ...     ...     ...     ...
245          1  196.73  171.64  160.62
246          1  178.00  196.73  171.64
247          1  211.72  178.00  196.73
248          1  212.36  211.72  178.00
249          1  266.86  212.36  211.72

[250 rows x 4 columns]
[[  1.        nan     nan     nan]
 [  1.    202.64      nan     nan]
 [  1.    166.81  202.64      nan]
 [  1.    181.09  166.81  202.64 ]
 [  1.    157.53  181.09  166.81 ]
 [  1.    170.84  157.53  181.09 ]
 [  1.    173.87  170.84  157.53 ]
 [  1.    146.98  173.87  170.84 ]
 [  1.    179.73  146.98  173.87 ]
 [  1.    139.06  179.73  146.98 ]
 [  1.    152.78  139.06  179.73 ]
 [  1.    161.09  152.78  139.06 ]
 [  1.    142.35  161.09  152.78 ]
 [  1.    176.25  142.35  161.09 ]
 [  1.    163.18  

Assign prediction interval that should be - 147

In [37]:
modelx = concatedarray[3:150] # New array for beats estimation
modely = yData[3:150]

print(modelx)

[[  1.    181.09  166.81  202.64 ]
 [  1.    157.53  181.09  166.81 ]
 [  1.    170.84  157.53  181.09 ]
 [  1.    173.87  170.84  157.53 ]
 [  1.    146.98  173.87  170.84 ]
 [  1.    179.73  146.98  173.87 ]
 [  1.    139.06  179.73  146.98 ]
 [  1.    152.78  139.06  179.73 ]
 [  1.    161.09  152.78  139.06 ]
 [  1.    142.35  161.09  152.78 ]
 [  1.    176.25  142.35  161.09 ]
 [  1.    163.18  176.25  142.35 ]
 [  1.     99.869 163.18  176.25 ]
 [  1.    148.7    99.869 163.18 ]
 [  1.    223.79  148.7    99.869]
 [  1.    157.91  223.79  148.7  ]
 [  1.    155.31  157.91  223.79 ]
 [  1.    107.48  155.31  157.91 ]
 [  1.    129.52  107.48  155.31 ]
 [  1.    155.36  129.52  107.48 ]
 [  1.    134.1   155.36  129.52 ]
 [  1.    149.43  134.1   155.36 ]
 [  1.    144.84  149.43  134.1  ]
 [  1.    143.71  144.84  149.43 ]
 [  1.    145.04  143.71  144.84 ]
 [  1.    139.8   145.04  143.71 ]
 [  1.    131.59  139.8   145.04 ]
 [  1.    161.55  131.59  139.8  ]
 [  1.    166.87  16

Estimate betas based on variable - modelx(matrix of all forecasts plus intersept) and modely(real values)

In [38]:
betas = np.dot(np.linalg.inv(np.dot(modelx.T, modelx)), np.dot(modelx.T, modely))
print(betas)

[ 6.31779473e+01  4.05625696e-01 -1.83955086e-02  2.49229234e-01]


Make forecast based on betas

In [39]:
modelx_forecast = concatedarray[151:]
forecast = np.dot(modelx_forecast, betas)
modely_evaluation = yData[151:]
print(forecast)

mae = np.mean(np.abs(forecast - modely_evaluation))

print(mae)

[216.95063703 217.78969122 226.54130507 221.96818918 231.81943528
 240.77950347 234.33399161 239.21514986 227.5966147  233.90833569
 228.68126065 234.19370552 218.37165764 216.78141821 217.49184824
 216.32517941 298.73831577 218.38542904 254.58413004 253.1151658
 195.65925816 223.70963133 213.63206967 202.25785843 222.27457428
 216.88614465 213.36418793 202.48615802 202.69866113 195.95815573
 194.4588456  195.67771794 194.50212671 190.21900211 198.64954591
 196.34103653 222.42510021 196.00739671 229.06689184 196.51524842
 230.02771083 222.5622656  227.27896605 232.57304774 222.81478823
 237.52935917 218.78605446 238.74903333 205.53992358 229.81180924
 217.7002612  232.26754857 226.70267458 240.74977487 232.09009249
 239.71980892 237.16402091 218.22479375 216.4263502  221.78810526
 217.85512297 223.31821716 212.24404313 215.6131088  206.58320281
 209.77093682 204.04594938 203.33735103 205.33446073 200.98190067
 203.6155546  193.38516617 186.77060487 175.68627109 178.34454464
 179.935323

# POINT (A)

Make average forecast of both

In [66]:
combined = (forecast + naive1['price'][151:])/2
print(combined)

151    224.980319
152    237.874846
153    237.085653
154    227.934095
155    246.509718
          ...    
245    170.235243
246    173.089039
247    195.771743
248    183.892363
249    216.001782
Name: price, Length: 99, dtype: float64


Calculate errors

In [74]:
err1 = yData[151:] - combined   # Calculate errors

coverages = []
nom_coverage = 0.5      # Define nom_coverage


quants = np.quantile(err1, [(1-nom_coverage)/2, (1+nom_coverage)/2])    # Quantile

low_PI = [ forec + quants[0] for forec in combined]   # lower prediction interval bound
up_PI = [ forec + quants[1] for forec in combined]    # upper prediction intrval bound

hits = []

print(low_PI)
print(up_PI)
print(yData[151:])

for day in range(len(low_PI)):
    if yData[151+day] < up_PI[day] and yData[151+day] > low_PI[day]:    # Check with real values if it fits into interval
        hits.append(1)
    else:
        hits.append(0)

print(hits)
print(hits.count(1)/len(hits))



[211.07749154387602, 223.97201864031143, 223.1828255658064, 214.03126762188538, 232.6068906695681, 236.28692476569, 236.7841688353575, 250.92974795849256, 235.4204803819482, 236.96134087553258, 225.91280335298103, 237.05902579183316, 228.2680018479162, 229.07288213423914, 210.53809714979673, 206.15476273626234, 259.67133091605245, 220.9498875517039, 333.1392380499384, 236.72975592854027, 190.47180210781585, 260.67698869580204, 198.13320786292178, 189.89610224453315, 222.69946017007453, 208.58524535675156, 216.9742669974715, 212.37025204150746, 201.93650359618272, 184.16625089265045, 189.5015958301792, 190.92603200119714, 184.7882363826775, 183.3966740845721, 189.60694598524287, 182.8026912962133, 204.75472313609782, 192.57587138311703, 235.83061894865483, 187.56979723944016, 227.20602844588097, 204.03830582764914, 232.76665605453212, 239.34869689800882, 224.25956714278794, 235.2618526146519, 220.3002002595174, 245.92668969332487, 210.3621348223386, 236.63307765156878, 201.9323036310224

# POINT (B) 

Calculate forecasts then make average of obtained lower and upper bounds from both forecasts. Then based on that calculate coverage.

MAKE UP AND LOW BOUND FOR NAIVE 

In [99]:
nom_coverage = 0.5      # Define nom_coverage

err2 = yData[151:] - naive1['price'][151:]  # Calculate errors
print(err2)
coverages = []

quants_naive1 = np.quantile(err2, [(1-nom_coverage)/2, (1+nom_coverage)/2])    # Quantile

low_PI_naive = [ forec2 + quants_naive1[0] for forec2 in naive1['price'][151:]]   # lower prediction interval bound
up_PI_naive = [ forec2 + quants_naive1[1] for forec2 in naive1['price'][151:]]    # upper prediction intrval bound

print(low_PI_naive)
print(up_PI_naive)

151     0.89
152     3.24
153    11.97
154    33.14
155    29.25
       ...  
245    17.38
246    40.08
247    15.63
248    88.86
249    20.81
Name: price, Length: 99, dtype: float64
[210.99999999999997, 235.94999999999996, 225.61999999999998, 211.89, 239.18999999999997, 237.59, 245.03, 268.43999999999994, 249.04, 245.80999999999997, 228.93999999999997, 245.72, 243.96, 247.16, 209.37999999999997, 201.77999999999997, 226.39999999999998, 229.30999999999997, 417.49, 226.14, 191.07999999999998, 303.43999999999994, 188.42999999999998, 183.32999999999998, 228.92, 206.07999999999998, 226.37999999999997, 228.04999999999998, 206.96999999999997, 178.17, 190.33999999999997, 191.96999999999997, 180.86999999999998, 182.36999999999998, 186.35999999999999, 175.05999999999997, 192.87999999999997, 194.93999999999997, 248.38999999999996, 184.42, 230.17999999999998, 191.30999999999997, 244.04999999999998, 251.92, 231.49999999999997, 238.79, 227.60999999999999, 258.9, 220.98, 249.24999999999997, 191.95999

MAKE UP AND LOW BOUND FOR REGRESSION

In [115]:
err3 = yData[151:] - forecast  # Calculate errors

coverages = []

quants_regress = np.quantile(err3, [(1-nom_coverage)/2, (1+nom_coverage)/2])    # Quantile

low_PI_regress = [ forec3 + quants_regress[0] for forec3 in forecast]   # lower prediction interval bound
up_PI_regress = [ forec3 + quants_regress[1] for forec3 in forecast]    # upper prediction intrval bound

print(low_PI_regress)
print(up_PI_regress)

[219.80118374875104, 220.64023794162182, 229.39185179261187, 224.81873590476977, 234.66998200013512, 243.63005019237903, 237.18453833171395, 242.06569657798406, 230.4471614248954, 236.7588824120641, 231.53180736696106, 237.04425224466536, 221.22220435683136, 219.63196492947725, 220.34239496059251, 219.1757261335237, 301.58886249310393, 221.23597576440676, 257.43467676087585, 255.9657125180795, 198.50980487663065, 226.56017805260308, 216.4826163868426, 205.10840515006532, 225.1251210011481, 219.73669137450213, 216.21473465594204, 205.3367047440139, 205.5492078533644, 198.8087024462999, 197.3093923213574, 198.52826466339332, 197.35267342635393, 193.06954883014322, 201.50009263148473, 199.19158325342562, 225.27564693319465, 198.85794342723307, 231.91743855830867, 199.36579513987937, 232.878257552761, 225.41281231629728, 230.1295127700633, 235.42359445701663, 225.66533494657494, 240.37990589030278, 221.63660118003378, 241.59958004764877, 208.39047030567616, 232.66235596413657, 220.55080792

NOW COMBINE TWO BOUNDS

In [116]:
# Calculate combined low and up bounds
combined_low_bound = (np.array(low_PI_naive) + np.array(low_PI_regress)) / 2
combined_up_bound = (np.array(up_PI_naive) + np.array(up_PI_regress)) / 2

hits_comb_2 = []

for day in range(len(combined_up_bound)):
    if yData[151+day] < combined_up_bound[day] and yData[151+day] > combined_low_bound[day]:    # Check with real values if it fits into interval
        hits_comb_2.append(1)
    else:
        hits_comb_2.append(0)

print(hits_comb_2)
print(hits_comb_2.count(1)/len(hits_comb_2))

[1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1]
0.494949494949495
